In [379]:
import pandas as pd
import os
import numpy as np
import json
import geopandas as gpd


In [380]:
old_url = 'http://seshat.datasd.org/pd/vehicle_stops_{}_datasd_v1.csv'
df1 =  pd.read_csv(old_url.format('2015'))

In [381]:
df1.columns

Index(['stop_id', 'stop_cause', 'service_area', 'subject_race', 'subject_sex',
       'subject_age', 'date_time', 'date_stop', 'time_stop', 'sd_resident',
       'arrested', 'searched', 'obtained_consent', 'contraband_found',
       'property_seized'],
      dtype='object')

In [382]:
df2 = pd.read_csv('http://seshat.datasd.org/pd/ripa_stops_datasd_v1.csv')

In [383]:
df2.columns

Index(['stop_id', 'ori', 'agency', 'exp_years', 'date_stop', 'time_stop',
       'stopduration', 'stop_in_response_to_cfs', 'officer_assignment_key',
       'assignment', 'intersection', 'address_block', 'land_mark',
       'address_street', 'highway_exit', 'isschool', 'school_name',
       'address_city', 'beat', 'beat_name', 'pid', 'isstudent',
       'perceived_limited_english', 'perceived_age', 'perceived_gender',
       'gender_nonconforming', 'gend', 'gend_nc', 'perceived_lgbt'],
      dtype='object')

#### What we want to keep as a general outline for data

stop_id

stop_cause / reason_for_stop

result_of_stop

stopdate

stoptime

service_area

block

subject_race / race --> map to race codes

beat (new) --> map to service area

actions_taken (new)

officer_assignment_key (new)

exp_years (new)

stopduration (new)

perceived_age (new)

In [384]:
# remove some
cols_keep = ['stop_id', 'date_stop', 'time_stop', 'stopduration', 
             'officer_assignment_key', 'exp_years', 'beat', 
             'perceived_age', 'gend']

In [385]:
basic = df2[cols_keep]
basic.head()

,stop_id,date_stop,time_stop,stopduration,officer_assignment_key,exp_years,beat,perceived_age,gend
0,2443,2018-07-01,00:01:37,30,1,10,122,25,1
1,2444,2018-07-01,00:03:34,10,1,18,121,25,1
2,2447,2018-07-01,00:05:43,15,10,1,822,30,1
3,2447,2018-07-01,00:05:43,15,10,1,822,30,2
4,2448,2018-07-01,00:19:06,5,1,3,614,23,1


In [386]:
web_base = 'https://data.sandiego.gov/datasets/?department=police'

reason = 'stop_reason'
reason_cols = ['stop_id', 'reason_for_stop', 'reason_for_stopcode']

result = 'stop_result'
result_cols= ['stop_id', 'result']

race = 'race'
race_cols = ['stop_id', 'race']

action = 'actions_taken'
action_cols = ['stop_id', 'action']

new_base = 'http://seshat.datasd.org/pd/ripa_{}_datasd.csv'

stop_reason = new_base.format(reason)
stop_result = new_base.format(result)
stop_race = new_base.format(race)
stop_action = 'http://seshat.datasd.org/pd/ripa_{}_datasd.csv'.format(action)

### columns we still need:

stop_cause / reason_for_stop, result_of_stop, actions_taken 

In [387]:
# using the columns from the tables below

In [388]:
sd_reason = new_base.format(reason)
reason_df =  pd.read_csv(sd_reason)

In [389]:
sd_result = new_base.format(result)
result_df =  pd.read_csv(sd_result)

In [390]:
sd_race = new_base.format(race)
race_df =  pd.read_csv(sd_race)

In [391]:
sd_action = new_base.format(action)
action_df =  pd.read_csv(sd_action)

In [392]:
def gen_cols(df1, gen_df, cols):
    new = df1.merge(gen_df, on = 'stop_id')
    drop = [x for x in new.columns if x not in cols]
    new = new.drop(columns = drop).drop_duplicates(subset = 'stop_id')
    return new

In [393]:
merge_cols = [reason_cols, result_cols, race_cols, action_cols]
merge_dfs = [reason_df, result_df, race_df, action_df]

merged = []
for i in range(len(merge_dfs)):
    merged.append(gen_cols(basic, merge_dfs[i], merge_cols[i]))

In [394]:
first = merged[0]

for i in range(0, len(merged)):
    if i == len(merged) - 1:
        break
    
    first = pd.merge(first, merged[i + 1], on = 'stop_id')
final = pd.merge(basic, first, on='stop_id').drop_duplicates(subset = 'stop_id')

In [395]:
final.head()

,stop_id,date_stop,time_stop,stopduration,officer_assignment_key,exp_years,beat,perceived_age,gend,reason_for_stop,reason_for_stopcode,result,race,action
0,2443,2018-07-01,00:01:37,30,1,10,122,25,1,Reasonable Suspicion,64005,Custodial Arrest without warrant,White,None
1,2444,2018-07-01,00:03:34,10,1,18,121,25,1,Traffic Violation,54106,Warning (verbal or written),White,None
2,2447,2018-07-01,00:05:43,15,10,1,822,30,1,Reasonable Suspicion,53072,No Action,Hispanic/Latino/a,Curbside detention
4,2448,2018-07-01,00:19:06,5,1,3,614,23,1,Traffic Violation,54106,No Action,White,None
5,2449,2018-07-01,00:03:00,15,1,1,115,25,1,Reasonable Suspicion,13045,Custodial Arrest without warrant,White,Curbside detention


### columns we still need to map:

subject_race / race - map to race codes

beat - map to service area

In [396]:
# engineered from existing file
# race_codes = 'http://seshat.datasd.org/pd/vehicle_stops_race_codes.csv'
# and intution

race_dict = {
 'Asian' : 'A',
 'OTHER ASIAN': 'A',
 'Middle Eastern or South Asian': 'M',
 'BLACK': 'B',
 'CHINESE': 'C',
 'CAMBODIAN': 'D',
 'FILIPINO': 'F',
 'GUAMANIAN': 'G',
 'HISPANIC': 'H',
 'Hispanic/Latino/a': 'H',
 'INDIAN': 'I',
 'JAPANESE': 'J',
 'KOREAN': 'K',
 'LAOTIAN': 'L',
 'OTHER': 'O',
 'PACIFIC ISLANDER': 'P',
 'Pacific Islander': 'P',
 'SAMOAN': 'S',
 'HAWAIIAN': 'U',
 'VIETNAMESE': 'V',
 'WHITE': 'W',
 'White': 'W',
 'ASIAN INDIAN': 'Z',
 'Native American': 'N'
}

In [397]:
final['Race Code'] = final['race'].map(race_dict)

In [398]:
stop_beats = 'http://seshat.datasd.org/sde/pd/pd_beats_datasd.geojson'

In [399]:
beats = gpd.read_file(stop_beats)

In [400]:
# get unique beats
uni = beats[['beat', 'serv']].drop_duplicates('beat')
beat_dict = dict(zip(uni.beat, uni.serv))

In [401]:
final['service_area'] = final['beat'].map(beat_dict)

In [402]:
final.columns

Index(['stop_id', 'date_stop', 'time_stop', 'stopduration',
       'officer_assignment_key', 'exp_years', 'beat', 'perceived_age', 'gend',
       'reason_for_stop', 'reason_for_stopcode', 'result', 'race', 'action',
       'Race Code', 'service_area'],
      dtype='object')

In [403]:
df1.columns

Index(['stop_id', 'stop_cause', 'service_area', 'subject_race', 'subject_sex',
       'subject_age', 'date_time', 'date_stop', 'time_stop', 'sd_resident',
       'arrested', 'searched', 'obtained_consent', 'contraband_found',
       'property_seized'],
      dtype='object')

### Clean column names

* These are the same: 'stop_id', 'date_stop', 'time_stop', 'service_area', 'subject_race'/'Race Code', 'subject_sex'/'gend',


* These are excess for post- 2018: 'beat', 'officer_assignment_key', 'exp_years', 'perceived_age'


* These are excess for pre- 2018: 
    - 'sd_resident'
    - 'arrested'
    - 'searched'
    - 'obtained_consent'
    - 'contraband_found'
    - 'property_seized'


&rightarrow;  Therefore: Need to make 'subject_race' == 'Race Code' and  'subject_sex' == 'gend'


* Additionally for pre- 2018, need to engineer: 'outcome'

    -  from ['arrested', 'searched', 'property_seized'] in pre- 2018 and ['action', 'result'] in post- 2018


* Columns we are ok with having lots of null values because they are useful for analysis:

    - 'sd_resident'
    - 'beat'
    - 'officer_assignment_key'
    - 'exp_years'
    - 'perceived_age'

In [404]:
final = final.rename(columns={'Race Code': 'subject_race', 'gend': 'subject_sex'})

In [405]:
# engineer action for pre- 2018
final.action.unique()

array(['None', 'Curbside detention', 'Handcuffed or flex cuffed',
       'Property was seized', 'Search of person was conducted',
       'Search of property was conducted',
       'Field sobriety test conducted', 'Chemical spray used',
       'Asked for consent to search person', 'Vehicle impounded',
       'Physical or Vehicle contact', 'Person photographed',
       'Patrol car detention', 'Asked for consent to search property',
       'Person removed from vehicle by order',
       'Canine removed from vehicle or used to search',
       'Baton or other impact weapon used', 'Firearm pointed at person',
       'Person removed from vehicle by physical contact',
       'Admission or written statement obtained from student',
       'Electronic control device used', 'Canine bit or held person',
       'Firearm discharged or used',
       'Impact projectile discharged or used'], dtype=object)

In [406]:
 final.result.unique()

array(['Custodial Arrest without warrant', 'Warning (verbal or written)',
       'No Action', 'Field interview card completed',
       'Citation for infraction',
       'Custodial Arrest pursuant to outstanding warrant',
       'Psychiatric hold', 'In-field cite and release',
       'Noncriminal transport or caretaking transport',
       'Contacted parent/legal guardian or other person responsible for the minor',
       'Contacted U.S. Department of Homeland Security',
       'Referral to school counselor or other support staff',
       'Referral to school administrator'], dtype=object)

In [407]:
# Outcome possible values: 

# 'None'
# 'Warning (verbal or written)' 
# 'Search of property was conducted' 
# 'Property was seized' >> property seized
# 'Custodial Arrest without warrant'

In [442]:
# helper functions for cleaning / merging
def c_bool(string):
    if (string == 'Y') | (string =='y'):
        return 1
    if (string == 'N') | (string == 'n'):
        return 0
    return np.nan

def make_bool(cols, df):
    for col in cols:
        if col not in list(df.columns):
            print(col)
            continue
        df[col] = df[col].apply(lambda x: c_bool(x))
    return df

In [444]:
df1.columns

Index(['stop_id', 'stop_cause', 'service_area', 'subject_race', 'subject_sex',
       'subject_age', 'date_time', 'date_stop', 'time_stop', 'sd_resident',
       'arrested', 'searched', 'obtained_consent', 'contraband_found',
       'property_seized'],
      dtype='object')

In [443]:
change_bool_old = ['sd_resident', 'search', 'contraband_found', 'property_seized', 'arrested']
df1 = make_bool(change_bool_old, df1)

search


In [439]:
df1

,stop_id,stop_cause,service_area,subject_race,subject_sex,subject_age,date_time,date_stop,time_stop,sd_resident,arrested,searched,obtained_consent,contraband_found,property_seized
0,1191287,Moving Violation,430,W,F,NaN,NaN,2015-01-01,NaN,NaN,NaN,N,NaN,NaN,NaN
1,1191295,Moving Violation,520,B,M,NaN,NaN,2015-01-01,NaN,NaN,NaN,N,NaN,NaN,NaN
2,1191275,Moving Violation,430,B,M,NaN,NaN,2015-01-01,NaN,NaN,NaN,N,NaN,NaN,NaN
3,1191308,Moving Violation,520,W,M,NaN,NaN,2015-01-01,NaN,NaN,NaN,N,NaN,NaN,NaN
4,1191285,Moving Violation,430,W,F,NaN,NaN,2015-01-01,NaN,NaN,NaN,N,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115417,1308165,Moving Violation,Unknown,H,F,38,2015-12-31 23:39:00,2015-12-31,23:39,NaN,NaN,N,NaN,NaN,NaN
115418,1308166,Equipment Violation,930,W,M,18,2015-12-31 23:40:00,2015-12-31,23:40,NaN,NaN,N,NaN,NaN,NaN
115419,1308168,Moving Violation,520,B,M,38,2015-12-31 23:52:00,2015-12-31,23:52,NaN,NaN,N,NaN,NaN,NaN
115420,1308169,Equipment Violation,830,B,F,33,2015-12-31 23:54:00,2015-12-31,23:54,NaN,NaN,NaN,NaN,NaN,NaN


In [422]:
df1.columns

Index(['stop_id', 'stop_cause', 'service_area', 'subject_race', 'subject_sex',
       'subject_age', 'date_time', 'date_stop', 'time_stop', 'sd_resident',
       'arrested', 'searched', 'obtained_consent', 'contraband_found',
       'property_seized'],
      dtype='object')

In [423]:
def check_outcome(row, year):
    print(row.arrested)
    if year < 2018:
        # check 'arrested', 'searched', 'property_seized'
        if row.arrested == 1:
            return 'Arrest'
        elif row.property_seized == 1:
            return 'Property was seized'
        elif row.searched == 1:
            return 'Search of property was conducted'
        return 'Not Applicable'
        
    else:
        a = row.action
        r = row.result
        
        # check worst outcome first
        if r == 'Custodial Arrest without warrant':
            return r
        elif (a == 'Search of property was conducted') | ('Property was seized'):
            return a
        elif r == 'Warning (verbal or written)':
            return r
        
        return 'Not Applicable'
    
def outcome_map(df, yr):
    df_copy = df.copy()
    df_copy['Outcome'] = df_copy.apply(lambda x: check_outcome(x, yr), axis=1)
        
    return df_copy

In [417]:
df1.columns

Index(['stop_id', 'stop_cause', 'service_area', 'subject_race', 'subject_sex',
       'subject_age', 'date_time', 'date_stop', 'time_stop', 'sd_resident',
       'arrested', 'searched', 'obtained_consent', 'contraband_found',
       'property_seized'],
      dtype='object')

In [419]:
df1.arrested.value_counts()

N    102965
n      2268
Y      1339
y         7
          6
M         1
b         1
Name: arrested, dtype: int64

In [418]:
t = outcome_map(df1, 2015)

N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
nan
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
nan
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N


N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
Y
N
N
nan
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
nan
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N


N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
Y
N
N
nan
nan
N
N
nan
nan
N
N
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
Y
N
N
N
N
N
N
N
N
Y
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
nan


N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
Y
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
nan
N
nan
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
nan
N
N
nan
N
nan
N
nan
Y
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N


N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
n
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
n
N
N
N
N
N
N
N
N
N
N
N
N
N
N
n
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N


...

N
N
N
N
N
N
N
N
N
N
N
nan
N
N
nan
N
N
N
nan
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
nan
N
nan
N
N
nan
N
N
nan
N
N
N
nan
N
N
N
nan
N
N
N
N
nan
N
N
N
nan
N
N
N
N
N
N
nan
N
N
N
N
N
nan
N
N
N
nan
N
N
N
nan
N
N
N
N
nan
N
N
N
nan
N
N
nan
N
nan
N
N
N
N
N
N
N
nan
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
nan
nan
N
N
N
N
nan
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
nan
N
N
N
Y
N
N
N
N
N
Y
N
N
N
nan
N
N
nan
N
N
N
N
N
N
N
Y
nan
N
N
N
nan
N
N
N
N
N
nan
N
N
N
N
N
N
N
nan
N
N
N
N
nan
N
N
nan
N
N
nan
nan
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
nan
N
N
N
N
N
nan
N
N
N
N
N
nan
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
nan
N
nan
N
N
N
N
nan
N
N
N
N
N
nan
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N


N
nan
N
N
N
N
nan
nan
N
nan
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
nan
N
N
N
nan
nan
nan
N
nan
N
nan
N
N
N
N
nan
nan
nan
N
N
N
N
N
N
N
nan
N
N
N
nan
N
N
N
nan
N
N
nan
N
N
nan
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
nan
N
N
N
N
N
N
N
N
N
N
nan
nan
N
nan
N
nan
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
Y
Y
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N


N
N
N
N
N
N
nan
N
nan
N
nan
N
N
N
N
nan
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
nan
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
nan
N
N
N
N
nan
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
nan
N
N
N
N
N
Y
nan
N
N
nan
N
N
nan
nan
N
nan
N
nan
nan
nan
N
nan
N
nan
N
nan
nan
nan
N
nan
nan
N
N
nan
N
N
nan
N
N
nan
N
N
N
N
N
nan
N
N
nan
N
N
nan
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
nan
N
nan
N
nan
N
N
Y
N
N
N
nan
N
N
N
N
N
N
nan
N
N
N


N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
nan
N
N
nan
N
nan
N
nan
N
nan
N
nan
nan
N
N
N
nan
Y
Y
N
nan
nan
N
N
N
nan
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
nan
nan
nan
N
nan
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
nan
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
nan
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
Y
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N


N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
nan
N
N
N
N
N
N
nan
nan
N
nan
nan
N
nan
N
N
nan
N
nan
N
N
N
N
nan
nan
nan
N
N
N
N
nan
nan
nan
N
N
N
nan
N
N
N
N
N
nan
N
N
N
N
nan
N
N
nan
N
N
N
nan
nan
N
N
N
N
nan
nan
N
nan
N
N
N
nan
nan
N
N
N
nan
N
nan
N
nan
N
N
N
N
N
N
N
N
N
nan
nan
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
nan
N
N
N
N
nan
N
N
N
N
N
Y
nan
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
N
N
N
N
N
N
N
N
N
N
N
nan
N
N
nan
N
N
N
N
N
N
N
nan
nan
N
N
N
N
N
nan
N
N
N
nan
N
nan
N
N
N
nan
N
N
N
Y
N
N
N
nan
N
N
N
N
N
N
N
nan
nan
N
nan
N
N
N
N
N
N
nan
N
N
N
N
N


In [415]:
t.columns

Index(['stop_id', 'stop_cause', 'service_area', 'subject_race', 'subject_sex',
       'subject_age', 'date_time', 'date_stop', 'time_stop', 'sd_resident',
       'arrested', 'searched', 'obtained_consent', 'contraband_found',
       'property_seized', 'Outcome'],
      dtype='object')

In [414]:
t.Outcome.value_counts()

Not Applicable         114932
Property was seized       490
Name: Outcome, dtype: int64

# Part 1

#### Cleaning
Perform an initial EDA to statistically assess the quality of the data and its appropriateness for addressing the problem at hand, justifying data cleaning logic. This will likely address issues with accuracy, precision, and missingness of specific attributes, tying these issues to their possible impact over eventual results.

#### Descriptive Stats
Statistically summarize the relevant, cleaned attributes and derived features (e.g. in univariate and bivariate analyses) for San Diego.

#### Traffic Stop Analysis
Calculate and document the differences in stop rates and post-stop outcomes. The analysis should address possible reasons for such differences (including addressing possible confounders). Additionally: - The significance of these differences should be tested using statistical inference. - These differences should also be calculated across other variables of interest (e.g. service area).

#### Veil of Darkness
Perform the Veil of Darkness analysis for San Diego. Include an introduction to the technique and interpret the results. (In Assignment #3 you will carefully address the shortcoming of this result.)

# Part 2

Develop code to clean data (as defined and justified in Part 1), create the features for the replication, and compute the statistics for the report. Such code should conform to the methodology portion of the course (e.g. using the project template).

In particular, your project should have a run.py with the following targets:

* data creates the data needed for analysis.
* process cleans and prepares the data for analysis (e.g. cleaning and feature creation).
* data-test ingests a small amount of test data (that process can then process).